# Implementation of Viterbi HMM POS tagger
author: Kailin Zheng<br>
date: October 1 2019<br>
class: HW3, NLP<br>

In [1]:
import nltk 
from nltk.tokenize import word_tokenize

In [2]:
import pandas as pd
import numpy as np

In [3]:
#read Development file (WSJ_24.pos)

#a list of sentence - each line has two lists: word and pos
sentences = [] 

word = []
pos = []


f= open("WSJ_24.pos","r+")


for line in f.readlines():
    words = line.split()
    
    if (words == []):
        sentences.append([word, pos])
        word = []
        pos = []
    else:
        word.append(words[0])
        pos.append(words[1])
f.close()  

f = open ("WSJ_02-21.pos","r+")

for line in f.readlines():
    words = line.split()
    
    if (words == []):
        sentences.append([word, pos])
        word = []
        pos = []
    else:
        word.append(words[0])
        pos.append(words[1])
f.close()


In [4]:
print(sentences[-1][1][5])

NN


In [5]:
observed_words = []
states = ['start state']
Once = {}
for i in range(len(sentences[:])):
    for j in range(len(sentences[i][0])):
        word = sentences[i][0][j]
        tag = sentences[i][1][j]
        if word in Once.keys():
            del Once[word]
        Once[word] = tag
        
        if (word not in observed_words):
            observed_words.append(word)
        if (tag not in states):
            states.append(tag)
states.append('final state')
print(states)
print(observed_words[:10])

['start state', 'DT', 'NN', 'POS', 'MD', 'VB', 'VBN', 'IN', 'JJ', 'NNS', ',', 'CC', '.', 'RBS', 'NNP', 'VBZ', 'TO', '$', 'CD', 'VBG', 'RB', 'VBD', 'PRP', 'PRP$', 'VBP', 'WRB', '``', 'EX', "''", ':', 'RBR', 'WP', 'JJR', 'WDT', 'RP', '#', 'PDT', 'UH', '(', ')', 'NNPS', 'JJS', 'SYM', 'FW', 'WP$', 'LS', 'final state']
['The', 'economy', "'s", 'temperature', 'will', 'be', 'taken', 'from', 'several', 'vantage']


In [6]:
OOV = pd.Series(0.0, index = states)
for v in Once.keys():
    OOV[Once[v]] += 1
    
OOV = OOV/len(Once)

In [7]:
#A = N ⨯ N matrix such that a_i,j is the probability of the transition from q_i to q_j
#B = lookup table such that bi(wt) is the probability that POS i is realized as word t
#q0 = start state, q1...qN, qf = final state

A = pd.DataFrame(0.0, index = states, columns = states)
B = pd.DataFrame(0.0, index = observed_words, columns = states)


for i in range(len(sentences[:])):
    prev = 'start state'
    for j in range(len(sentences[i][1])):
        now = sentences[i][1][j]
        A.loc[prev,now] += 1
        prev = now
        
        B.loc[sentences[i][0][j], sentences[i][1][j]] += 1

A = A.div(A.sum(axis=1), axis=0)
B = B.div(B.sum(axis=1), axis=0)
B

,start state,DT,NN,POS,MD,VB,VBN,IN,JJ,NNS,...,UH,(,),NNPS,JJS,SYM,FW,WP$,LS,final state
The,0.0,0.994325,0.000000,0.000000,0.000000,0.000142,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
economy,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
's,0.0,0.000000,0.000000,0.867539,0.000000,0.000000,0.0,0.000000,0.000000,0.000104,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
temperature,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
will,0.0,0.000000,0.006923,0.000000,0.992763,0.000315,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
be,0.0,0.000000,0.000000,0.000000,0.000000,0.999743,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
taken,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
from,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
several,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
vantage,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [8]:
A

,start state,DT,NN,POS,MD,VB,VBN,IN,JJ,NNS,...,UH,(,),NNPS,JJS,SYM,FW,WP$,LS,final state
start state,0.0,0.216353,0.039924,0.000000,0.000777,0.002987,0.005756,0.127204,0.041770,0.041843,...,0.000631,0.003667,0.000024,0.001991,0.002428,0.001360,0.000170,0.000024,0.000777,0.0
DT,0.0,0.001535,0.474206,0.000035,0.002161,0.000248,0.008490,0.009741,0.217922,0.073420,...,0.000024,0.000508,0.000024,0.004971,0.009293,0.000000,0.000260,0.000000,0.000000,0.0
NN,0.0,0.006790,0.122329,0.021775,0.017699,0.001434,0.010691,0.247671,0.008726,0.078030,...,0.000000,0.001616,0.001790,0.000095,0.000044,0.000029,0.000051,0.000196,0.000000,0.0
POS,0.0,0.000222,0.418778,0.000000,0.001111,0.000333,0.007333,0.002778,0.206889,0.129111,...,0.000000,0.000222,0.000889,0.003778,0.025222,0.000000,0.000222,0.000000,0.000000,0.0
MD,0.0,0.003944,0.000789,0.000000,0.000197,0.797279,0.000493,0.001873,0.000493,0.000690,...,0.000000,0.000197,0.000197,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
VB,0.0,0.220894,0.061582,0.000182,0.000474,0.005029,0.084211,0.112451,0.084867,0.050432,...,0.000109,0.001093,0.000437,0.000510,0.000692,0.000000,0.000073,0.000036,0.000000,0.0
VBN,0.0,0.068005,0.065550,0.000144,0.001155,0.000722,0.032101,0.365242,0.043411,0.038262,...,0.000000,0.000481,0.000866,0.000193,0.000193,0.000000,0.000000,0.000000,0.000000,0.0
IN,0.0,0.328294,0.108733,0.000020,0.000108,0.000480,0.004435,0.020717,0.090258,0.058753,...,0.000010,0.000333,0.000078,0.001860,0.004670,0.000020,0.000196,0.000020,0.000010,0.0
JJ,0.0,0.003494,0.449841,0.000190,0.000411,0.000111,0.001881,0.055558,0.074215,0.232035,...,0.000016,0.000680,0.000569,0.001739,0.000348,0.000000,0.000126,0.000000,0.000000,0.0
NNS,0.0,0.016881,0.020891,0.009136,0.028022,0.003945,0.021182,0.235075,0.016622,0.010478,...,0.000000,0.004398,0.001649,0.000032,0.000032,0.000016,0.000000,0.000420,0.000000,0.0


In [9]:
#read test file (WSJ_23.pos)

#a list of sentence - each line has two lists: word and pos
S = [] 

word = []
pos = []
f= open("WSJ_23.words","r+")

for line in f.readlines():
    words = line.split()
    
    if (words == []):
        S.append([word, pos])
        word = []
        pos = []
    else:
        word.append(words[0])
    
print(S[1])

[['But', 'while', 'the', 'New', 'York', 'Stock', 'Exchange', 'did', "n't", 'fall', 'apart', 'Friday', 'as', 'the', 'Dow', 'Jones', 'Industrial', 'Average', 'plunged', '190.58', 'points', '--', 'most', 'of', 'it', 'in', 'the', 'final', 'hour', '--', 'it', 'barely', 'managed', 'to', 'stay', 'this', 'side', 'of', 'chaos', '.'], []]


In [10]:
def viterbi_algo(A,B,S):
    result = []
    for i in range(len(S)):
        if i % 50 == 0:
            print(i)
        
        

        viterbi = pd.DataFrame(0.0, index = states, columns = S[i][0])
        viterbi.insert(0, column = '\\start', value = np.zeros(len(states)))
        viterbi.insert(len(viterbi.columns), column = '\\end', value = 0.0)
        viterbi.loc['start state', '\\start'] = 1
        backpointer = pd.Series(-1, index = viterbi.columns)
        
        #initialize all states of the first word in viterbi: aka column[1]
        for q in range(1, len(states)):
            state = states[q]
            first_word = S[i][0][0]
            if (first_word not in observed_words):
                viterbi.iloc[q,1] = A.iloc[0,q] * OOV[states[q]]
            else:
                viterbi.iloc[q,1] = A.iloc[0,q] * B.loc[first_word, state]
            backpointer[1] = 0
        #print(viterbi)
        
        #w in viterbi, starting 1 = w-1 in sentence, starting 0
        for w in range(1, len(viterbi.columns)-1):
            #print(i,',',w)
            word = S[i][0][w-1]
            score = 0
            for q in range(len(states)): #state of prev word
                if viterbi.iloc[q][w-1] == 0:
                    continue
                for p in range(len(states)): #state of curr word
                    curr_state = states[p]
                    if word not in observed_words:
                        score = viterbi.iloc[q, w-1] * A.iloc[q,p] * OOV[states[q]]
                    else:
                        if B.loc[word, curr_state] == 0:
                            continue
                        score = viterbi.iloc[q, w-1] * A.iloc[q,p] * B.loc[word, curr_state]
                    if score > viterbi.iloc[p,w]:
                        viterbi.iloc[p,w] = score
                        backpointer[w] = q
        

        max_index = -1
        for q in range(len(states)):
            if viterbi.iloc[q,-2] > viterbi.iloc[-1,-1]:
                max_index = q
                viterbi.iloc[-1,-1] = viterbi.iloc[q,-2]
                backpointer[-1] = q
                

        #print(viterbi)
        #print(backpointer)
        result.append(backpointer)
    return result

In [11]:
backpointers = viterbi_algo(A,B,S)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400


In [12]:
result = open("submission.pos","w+")
for i in range(len(S)):

    b = backpointers[i]
    for w in range(len(S[i][0])):
        result.write(S[i][0][w]+'\t'+states[b[w+2]])
        result.write('\n')
    result.write('\n')
result.close()